# Manipulation of trajectory data for rough alignment of Sparse point cloud

##### Here we calculate the length of the scanning-trajectory and divide it into sub intervals based on the Image scanning frequency and calculate normals to initialize rotation parameters of sparse point cloud from each image in the sequence

In [2]:
import math
import numpy as np
import open3d as o3d

In [10]:
def distance(x1,x2,y1,y2):
    '''
    This function is created to calculate distances between points of the trajectory.
    As most of the points lie on the same horizontal plane, there isn't much change 
    in the z values of the points. Thus, distance is calculated in the horizontal plane
    itself using the x and y coordinates
    '''
    return math.sqrt(((x2-x1)**2)+((y2-y1)**2))

def traj_len(traj, resolution):
    """
    traj is an array with 3 columns for x,y,z coordinates
    resolution is a positive integer defining no. of points to skip
    Calculates the length of the trajectory in the horizontal plane
    with a given resolution of no. of consecutive points along the
    trajectory
    """
    length=0
    r=resolution
    print(len(traj))
    for i in range(0, len(traj)-r, r):
        x1=traj[i][0]
        x2=traj[i+r][0]
        y1=traj[i][1]
        y2=traj[i+r][1]
        length+=distance(x1,x2,y1,y2)
        
    return length


In [4]:
traj=np.loadtxt("Data/test_traj.txt")
length_of_traj=traj_len(traj, 100)
print(length_of_traj)

17523
140.7503984436537


We have 228 images in the total dataset so length of sub-interval = 140/228 = 0.61 meters (approximately)

### Subinterval calculator

In [27]:
def sub_calc(traj, interval_length):
    subintervals=[]
    sub_length=0
    subintervals.append(traj[0])
    for i in range(0, len(traj)-1):
        x1=traj[i][0]
        x2=traj[i+1][0]
        y1=traj[i][1]
        y2=traj[i+1][1]
        sub_length+=distance(x1,x2,y1,y2)
        if(sub_length>interval_length):
            subintervals.append(traj[i+1])
            sub_length=0
    return subintervals

In [32]:
subs=sub_calc(traj,0.32) #adjusting interval length according to our dataset
intervals=np.array(subs)

In [33]:
intervals.shape

(438, 3)

In [34]:
#np.savetxt("Data/test_intervals.txt",intervals)

### Normal direction calculator

Calculating rotational angle only about z-axis

In [39]:
def rotation_calc(intervals):
    angles=[]
    for j in range(len(intervals)-1):
        x=intervals[j+1,0]-intervals[j,0]
        y=intervals[j+1,1]-intervals[j,1]
        angles.append(math.atan(y/x))
    return angles

In [43]:
rotation_z=np.array(rotation_calc(intervals))
rotation_z.shape

(437,)

In [45]:
rotation_z

array([ 1.18079554e+00,  1.42653415e+00,  1.56579712e+00,  1.49417954e+00,
       -1.56654531e+00, -1.53513154e+00,  1.54454652e+00, -1.56206091e+00,
       -1.50343249e+00, -1.54100365e+00, -1.53399113e+00,  1.56896203e+00,
        1.49785868e+00,  1.53020008e+00,  1.46675015e+00, -1.56703598e+00,
       -1.53921554e+00,  1.53596577e+00, -1.50369718e+00, -1.30366831e+00,
       -1.12449971e+00, -8.56210221e-01, -4.53953552e-01, -2.99410248e-01,
       -1.11684774e-01,  4.62385443e-02, -8.12364393e-03, -1.11640727e-01,
       -4.63554086e-02, -6.08470100e-02, -1.54618524e-01, -3.39295185e-02,
       -1.27105797e-01, -2.20461106e-01, -3.81156050e-02, -1.75158841e-02,
       -9.09596536e-02, -3.87436013e-02,  3.70304139e-02,  3.26388223e-02,
        2.58326538e-01,  4.09652701e-01,  5.47327251e-01,  1.13534056e+00,
        1.47755729e+00,  1.40985473e+00, -1.51872285e+00,  1.47766169e+00,
        1.46010437e+00,  1.51286416e+00,  1.45097272e+00,  1.47268729e+00,
        1.53140546e+00,  

### Manual Rotations

Estimating rotations along z-axis manually by observing the image dataset

In [59]:
_1=[0]*10
_2=[-1*np.pi/6]*16
_3=[0]*5
_4=[np.pi/4]*5
_5=[np.pi/2]*5
_6=[np.pi*0.75]*22
_7=[np.pi]*5
_8=[np.pi*1.5]*6

manual_rotations=_1+_2+_3+_4+_5+_6+_7+_8

In [79]:
angles=np.array(manual_rotations)
rot_z=np.zeros((74,2),dtype=float)
rotations=np.column_stack((rot_z,angles))

np.savetxt("Data/rotations.txt",rotations)